In [1]:
from feel_it import EmotionClassifier, SentimentClassifier
import pandas as pd
import numpy as np
import csv
import ast
from unidecode import unidecode
import re

c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
path = "E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo\Facebook\csv_docs_def\comments4_no_spam_nan.csv"
comments_no_nan = pd.read_csv(path, parse_dates=["post_date"], quotechar='"', quoting=csv.QUOTE_ALL, encoding="utf-8", low_memory=False)

In [3]:
path = "E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo\Facebook\csv_docs_def\posts4.csv"
posts = pd.read_csv(path, parse_dates=["post_date"], quotechar='"', quoting=csv.QUOTE_ALL, encoding="utf-8", low_memory=False)

In [4]:
def replace_dots(text):
    # This pattern will match sequences of words separated by dots but not acronyms.
    # Acronyms are sequences of single uppercase letters separated by dots and potentially followed by a dot.
    #pattern = r'\b(?:\w+\.\w{2,}|\w{2,}\.\w+)(?:\.\w+)*\b'
    pattern = r'\b(?:[a-zA-Z]+\.[a-zA-Z]{2,}|[a-zA-Z]{2,}\.[a-zA-Z]+)(?:\.[a-zA-Z]+)*\b'
    #pattern = r'\b(?!\d{1,2}\.\d{1,2}\.\d{2,4}\b)(?=\w*[^\d]\w*)(?:\w{2,}\.\w+|\w+\.\w{2,})(?:\.\w+)*\b'

    def replacement(match):
        # Replace dots with spaces in the matched string
        return match.group(0).replace('.', '. ')
    
    # Use re.sub to replace the matched pattern with the replacement function
    return re.sub(pattern, replacement, text)

def replace_commas(text):
    # This pattern will match sequences of words separated by dots but not acronyms.
    # Acronyms are sequences of single uppercase letters separated by dots and potentially followed by a dot.
    #pattern = r'\b(?:\w+\.\w{2,}|\w{2,}\.\w+)(?:\.\w+)*\b'
    pattern = r'\b(?:[a-zA-Z]+,[a-zA-Z]{2,}|[a-zA-Z]{2,},[a-zA-Z]+)(?:,[a-zA-Z]+)*\b'
    #pattern = r'\b(?!\d{1,2}\.\d{1,2}\.\d{2,4}\b)(?=\w*[^\d]\w*)(?:\w{2,}\.\w+|\w+\.\w{2,})(?:\.\w+)*\b'

    def replacement(match):
        # Replace dots with spaces in the matched string
        return match.group(0).replace(',', ', ')
    
    # Use re.sub to replace the matched pattern with the replacement function
    return re.sub(pattern, replacement, text)

def clean_text(text):
    # Rimuove gli articoli con apostrofi come "l'", "all'", ecc.
    cleaned_text = re.sub(r"^(l|all|d|nell|sull|coll|dall|dell|all|quell|quest)'", '', text)
    return cleaned_text

def remove_leading_point(text):
    # Define a pattern to match words starting with a point
    #pattern = r'\W\.\b(?=\w+)'
    pattern = r'(?:(?<=\s)|(?<=^))\.\b(?=\w+)'

    # Use re.sub to remove the leading point from matched words
    result = re.sub(pattern, '', text)

    return result

def remove_ending_point(text):
    # Define a pattern to match words starting with a point
    #pattern = r'\W\.\b(?=\w+)'
    pattern = r'(?:\b)\.$'

    # Use re.sub to remove the leading point from matched words
    result = re.sub(pattern, '', text)

    return result

def remove_hyphen(text):
    # Define a pattern to match words starting with a point
    #pattern = r'\W\.\b(?=\w+)'
    pattern = r'(?<=[a-zA-Z])-(?=[a-zA-Z])'

    # Use re.sub to remove the leading point from matched words
    result = re.sub(pattern, ' ', text)

    return result

def remove_link(text):   
    
    pattern = r'\bhttps?\S+|\bcom/share\S+|\bstory_fbid=\S+|\bcom/story\S+'
    result = re.sub(pattern, '', text)

    return result

def transform_text(text):
    text = str(text)
    #text = str(text).lower()
    text = unidecode(text)
    text = replace_dots(text)
    text = replace_commas(text)
    text = remove_leading_point(text)
    text = remove_hyphen(text)
    text = remove_link(text)
    return text

In [5]:
def remove_receiver(row):
    if row["level"] > 0 and isinstance(row["content"], str):
        receiver_name_len = len(row["type"].split()) - len(row["author"].split()) - 7
        if len(row["content"].split()) >= receiver_name_len:
            token_list = row["content"].split()
            del token_list[:receiver_name_len]
            content_without_receiver = " ".join(token_list)
            row["content"] = content_without_receiver
    return row

# Apply the function to each row
comments_truncated = comments_no_nan.apply(remove_receiver, axis=1)
comments_truncated = comments_truncated.set_index('index')

In [6]:
comments_truncated

,comment_id,post_date,post_date_count,comment_date,author,content,likes,level,type,time_of_fetching,author_url,sex
index,,,,,,,,,,,,
0,0,2024-04-20,1,2w,Leo Sileo,Il clima viene controllato con la tecnologia,7,0.0,Comment by Leo Sileo 2 weeks ago,2024-05-06 11:30:55.757595,NaN,0
1,1,2024-04-20,1,2w,Luigi Sposato,Yes tutto una manipolazione del clima adesso ...,1,0.0,Comment by Luigi Sposato 2 weeks ago,2024-05-06 11:30:55.757595,NaN,0
2,2,2024-04-20,1,2w,Maccioni Bruno,Fresco? C'è il rischio che non rimanga nessun ...,21,0.0,Comment by Maccioni Bruno 2 weeks ago,2024-05-06 11:30:55.773181,NaN,0
3,3,2024-04-20,1,2w,Giuseppe Quattromini,colpa del riscaldamento globale,3,1.0,Reply by Giuseppe Quattromini to Maccioni Brun...,2024-05-06 11:30:55.773181,NaN,0
4,4,2024-04-20,1,2w,Maccioni Bruno,"Questo io non sono in grado di stabilirlo, pos...",5,2.0,Reply by Maccioni Bruno to Giuseppe Quattromin...,2024-05-06 11:30:55.788802,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
162895,38,2023-11-01,2,26w,Luciana Parodi,Invece sei bellissima e genuina,0,0.0,Comment by Luciana Parodi 26 weeks ago,2024-05-06 03:49:16.000957,NaN,1
162896,39,2023-11-01,2,26w,Franco Scaccia,E fai bene,0,0.0,Comment by Franco Scaccia 26 weeks ago,2024-05-06 03:49:16.000957,NaN,0
162897,40,2023-11-01,2,25w,Carlo Iaconi,Complimenti sei fantastica devi essere fiera d...,0,0.0,Comment by Carlo Iaconi 25 weeks ago,2024-05-06 03:49:16.007462,NaN,0


input_file = "data.csv"
output_file = "data_modified.csv"

# Read the CSV file and store rows in a list
with open(input_file, mode='r', encoding='utf-8', newline='') as infile:
    reader = csv.DictReader(infile)
    rows = list(reader)

# Modify the 'age' column for each row
for row in rows:
    row['age'] = int(row['age']) + 1

# Write the modified rows back to a new CSV file
with open(output_file, mode='w', encoding='utf-8', newline='') as outfile:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(rows)

In [7]:
'''def def_value(): 
    return 0

entities_freq = defaultdict(def_value)
entities_flag = {}
entities_link_prob = set()'''
count = 0
input_file = "E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo/Facebook/csv_docs_def/comments_entities.csv"
input_file2 = "E:/Gianluca/Master Big Data Pisa/Progetto_Finale/Agricolo/Facebook/csv_docs_def/posts4.csv"
post_information = []
interesting_comments = []
with open(input_file2, 'r', encoding='utf-8', newline='') as handle_r2:
    with open(input_file, 'r', encoding='utf-8', newline='') as handle_r:
        reader = csv.DictReader(handle_r)
        for row in reader:
            row_entities = ast.literal_eval(row["entities"])
            if len(row_entities) == 0:
                continue
            else:
                for entity in row_entities:
                    try:
                        if entity["title"] == "Europa" or entity["title"] == "Unione europea":
                            count = count + 1
                            interesting_comments.append(transform_text(comments_truncated.iloc[int(row["index"])]["content"]))
                            date_and_count = list(comments_truncated.iloc[int(row["index"])]["post_date", "post_date_count"])
                            link = posts[posts["post_date", "post_date_count"] == date_and_count]["url"]
                            post_information.append(date_and_count.append(link))
                            break
                    except KeyError:
                        pass
    

In [8]:
# interesting_comments

In [9]:
len(interesting_comments)

4782

In [10]:
count

4782

In [11]:
emotion_classifier = EmotionClassifier()
sentiment_classifier = SentimentClassifier()

c:\Users\giang\anaconda3\envs\env_master\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [12]:
# europa_emotions = emotion_classifier.predict(interesting_comments)

In [13]:
# np.save(r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\europa_emotions.npy", np.array(europa_emotions))
europa_emotions = np.load(r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\europa_emotions.npy")

In [14]:
np.unique(np.array(europa_emotions), return_counts=True)

(array(['anger', 'fear', 'joy', 'sadness'], dtype='<U7'),
 array([4192,   68,  175,  347], dtype=int64))

In [15]:
# europa_sentiments = sentiment_classifier.predict(interesting_comments)

In [16]:
# np.save(r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\europa_sentiments.npy", np.array(europa_sentiments))
europa_sentiments = np.load(r"E:\Gianluca\Master Big Data Pisa\Progetto_Finale\Agricolo 2\Group_6_2024_Project\Facebook\europa_sentiments.npy")

In [17]:
np.unique(np.array(europa_sentiments), return_counts=True)

(array(['negative', 'positive'], dtype='<U8'),
 array([4585,  197], dtype=int64))